In [35]:
import sklearn
# fetching a dataset of handwritten numbers
from sklearn.datasets import fetch_mldata
# Scikit-Learn provides many helper functions to download popular datasets. MNIST is one of them
mnist = fetch_mldata('MNIST original')

/home/ansh/Desktop/ml/handson/env/lib/python3.6/site-packages/sklearn/utils/deprecation.py:85: DeprecationWarning: Function fetch_mldata is deprecated; fetch_mldata was deprecated in version 0.20 and will be removed in version 0.22. Please use fetch_openml.
  warnings.warn(msg, category=DeprecationWarning)
/home/ansh/Desktop/ml/handson/env/lib/python3.6/site-packages/sklearn/utils/deprecation.py:85: DeprecationWarning: Function mldata_filename is deprecated; mldata_filename was deprecated in version 0.20 and will be removed in version 0.22. Please use fetch_openml.
  warnings.warn(msg, category=DeprecationWarning)


In [8]:
print(mnist)

{'DESCR': 'mldata.org dataset: mnist-original', 'COL_NAMES': ['label', 'data'], 'target': array([0., 0., 0., ..., 9., 9., 9.]), 'data': array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)}


In [9]:
X, y = mnist["data"], mnist["target"]

In [13]:
X.shape

(70000, 784)

In [12]:
y.shape

(70000,)

In [25]:
X_train, X_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]
import numpy as np
shuffle_index = np.random.permutation(60000)
X_train, y_train = X_train[shuffle_index], y_train[shuffle_index]

In [27]:
# Let’s simplify the problem for now and only try to identify one digit — for example, the number 5. This
# “5-detector” will be an example of a binary classifier, capable of distinguishing between just two
# classes, 5 and not-5. Let’s create the target vectors for this classification task:

In [28]:
y_train_5 = (y_train == 5)
y_test_5 = (y_test == 5)
# True for all 5s, False for all other digits.

In [29]:
# # Okay, now let’s pick a classifier and train it. A good place to start is with a Stochastic Gradient Descent
# (SGD) classifier, using Scikit-Learn’s SGDClassifier class. This classifier has the advantage of being
# capable of handling very large datasets efficiently. This is in part because SGD deals with training
# instances independently, one at a time (


# which also makes SGD well suited for online learning), as we



# will see later. Let’s create an SGDClassifier and train it on the whole training set:

In [33]:
from sklearn.linear_model import SGDClassifier
sgd_clf = SGDClassifier(random_state=42)
sgd_clf.fit(X_train, y_train_5)

ImportError: cannot import name 'signature'

In [37]:
#Testing on a cross validation sets
from sklearn.model_selection import cross_val_score
cross_val_score(sgd_clf, X_train, y_train_5, cv=3)

ImportError: cannot import name 'signature'

In [38]:
# CONFUSION MATRIX

# To compute the confusion matrix, you first need to have a set of predictions, so they can be compared to
# the actual targets. You could make predictions on the test set, but let’s keep it untouched for now
# (remember that you want to use the test set only at the very end of your project, once you have a classifier
# that you are ready to launch). Instead, you can use the cross_val_predict() function:
from sklearn.model_selection import cross_val_predict
y_train_pred = cross_val_predict(sgd_clf, X_train, y_train_5, cv=3)
from sklearn.metrics import confusion_matrix
confusion_matrix(y_train_5, y_train_pred)
# array([[53272, 1307],
# [ 1077, 4344]])

ImportError: cannot import name 'signature'

In [39]:
# CALCULATING PREDICTION AND RECALL

from sklearn.metrics import precision_score, recall_score
precision_score(y_train_5, y_train_pred) # == 4344 / (4344 + 1307)
# 0.76871350203503808
recall_score(y_train_5, y_train_pred) # == 4344 / (4344 + 1077)
# 0.80132816823464303

NameError: name 'y_train_pred' is not defined

In [1]:
# The F 1 score is the harmonic mean of precision and recall (Equation 3-3). Whereas the regular mean treats all values equally, the harmonic
# mean gives much more weight to low values. As a result, the classifier will only get a high F 1 score if
# both recall and precision are high.
from sklearn.metrics import f1_score
f1_score(y_train_5, y_train_pred)
# 0.78468208092485547

NameError: name 'y_train_5' is not defined

In [2]:
# To set a custom Threshold for the scores (cost function)
# ou will first need to get the scores of all
# instances in the training set using the cross_val_predict() function again, but this time specifying that
# you want it to return decision scores instead of predictions:

y_scores = cross_val_predict(sgd_clf, X_train, y_train_5, cv=3,
method="decision_function")

# Now with these scores you can compute precision and recall for all possible thresholds using the
# precision_recall_curve() function:
from sklearn.metrics import precision_recall_curve
precisions, recalls, thresholds = precision_recall_curve(y_train_5, y_scores)

# Finally, you can plot precision and recall as functions of the threshold value using Matplotlib
def plot_precision_recall_vs_threshold(precisions, recalls, thresholds):
    plt.plot(thresholds, precisions[:-1], "b--", label="Precision")
    plt.plot(thresholds, recalls[:-1], "g-", label="Recall")
    plt.xlabel("Threshold")
    plt.legend(loc="upper left")
    plt.ylim([0, 1])
    plot_precision_recall_vs_threshold(precisions, recalls, thresholds)
    plt.show()

y_train_pred_90 = (y_scores > 70000)
precision_score(y_train_5, y_train_pred_90)
recall_score(y_train_5, y_train_pred_90)

NameError: name 'cross_val_predict' is not defined

In [3]:
# The receiver operating characteristic (ROC) curve is another common tool used with binary classifiers.
# It is very similar to the precision/recall curve, but instead of plotting precision versus recall, the ROC
# curve plots the true positive rate (another name for recall) against the false positive rate. The FPR is the
# ratio of negative instances that are incorrectly classified as positive. It is equal to one minus the true
# negative rate, which is the ratio of negative instances that are correctly classified as negative. The TNR
# is also called specificity. Hence the ROC curve plots sensitivity (recall) versus 1 – specificity.
# To plot the ROC curve, you first need to compute the TPR and FPR for various threshold values, using the
# roc_curve() function:

from sklearn.metrics import roc_curve
fpr, tpr, thresholds = roc_curve(y_train_5, y_scores)

# Then you can plot the FPR against the TPR using Matplotlib. This code produces the plot in Figure 3-6:

def plot_roc_curve(fpr, tpr, label=None):
    plt.plot(fpr, tpr, linewidth=2, label=label)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.axis([0, 1, 0, 1])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
plot_roc_curve(fpr, tpr)
plt.show()

NameError: name 'y_train_5' is not defined

In [4]:
# Whereas binary classifiers distinguish between two classes, multiclass classifiers (also called
# multinomial classifiers) can distinguish between more than two classes.
# Some algorithms (such as Random Forest classifiers or naive Bayes classifiers) are capable of handling
# multiple classes directly. Others (such as Support Vector Machine classifiers or Linear classifiers) are
# strictly binary classifiers. However, there are various strategies that you can use to perform multiclass
# classification using multiple binary classifiers.
# Some algorithms (such as Support Vector Machine classifiers) scale poorly with the size of the training
# set, so for these algorithms OvO is preferred since it is faster to train many classifiers on small training
# sets than training few classifiers on large training sets. For most binary classification algorithms,
# however, OvA is preferred.


# Let’s try the SGDClassifier :
sgd_clf.fit(X_train, y_train) # y_train, not y_train_5
sgd_clf.predict([some_digit])
# Output-
# array([ 5.])


# That was easy! This code trains the SGDClassifier on the training set using the original target classes
# from 0 to 9 ( y_train ), instead of the 5-versus-all target classes ( y_train_5 ). Then it makes a prediction
# (a correct one in this case). Under the hood, Scikit-Learn actually trained 10 binary classifiers, got their
# decision scores for the image, and selected the class with the highest score.
# To see that this is indeed the case, you can call the decision_function() method. Instead of returning
# just one score per instance, it now returns 10 scores, one per class:
some_digit_scores = sgd_clf.decision_function([some_digit])
some_digit_scores
# array([[-311402.62954431, -363517.28355739, -446449.5306454 ,-183226.61023518, -414337.15339485, 161855.74572176,
# -452576.39616343, -471957.14962573, -518542.33997148,
# -536774.63961222]])

NameError: name 'sgd_clf' is not defined

In [ ]:
# MULTICLASS CLASSIFICATION


# If you want to force ScikitLearn to use one-versus-one or one-versus-all, you can use the
# OneVsOneClassifier or OneVsRestClassifier classes. Simply create an instance and pass a binary
# classifier to its constructor. For example, this code creates a multiclass classifier using the OvO strategy,
# based on a SGDClassifier :

from sklearn.multiclass import OneVsOneClassifier
ovo_clf = OneVsOneClassifier(SGDClassifier(random_state=42))
ovo_clf.fit(X_train, y_train)
ovo_clf.predict([some_digit])
# array([ 5.])
len(ovo_clf.estimators_)
# 45

# Training a RandomForestClassifier is just as easy:

forest_clf.fit(X_train, y_train)
forest_clf.predict([some_digit])
# array([ 5.])

# This time Scikit-Learn did not have to run OvA or OvO because Random Forest classifiers can directly
# classify instances into multiple classes. You can call predict_proba() to get the list of probabilities that
# the classifier assigned to each instance for each class:

forest_clf.predict_proba([some_digit])
# array([[ 0.1, 0. , 0. , 0.1, 0. , 0.8,
# 0. ,
# 0. ,
# 0. ,
# 0. ]])

# You can see that the classifier is fairly confident about its prediction: the 0.8 at the 5 th index in the array
# means that the model estimates an 80% probability that the image represents a 5. It also thinks that the
# image could instead be a 0 or a 3 (10% chance each).
# Now of course you want to evaluate these classifiers. As usual, you want to use cross-validation. Let’s
# evaluate the SGDClassifier ’s accuracy using the cross_val_score() function:>>> cross_val_score(sgd_clf, X_train, y_train, cv=3, scoring="accuracy")
# array([ 0.84063187, 0.84899245, 0.86652998])
# It gets over 84% on all test folds. If you used a random classifier, you would get 10% accuracy, so this is
# not such a bad score, but you can still do much better. For example, simply scaling the inputs (as
# discussed in Chapter 2) increases accuracy above 90%:

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train.astype(np.float64))
ross_val_score(sgd_clf, X_train_scaled, y_train, cv=3, scoring="accuracy")
# array([ 0.91011798, 0.90874544, 0.906636 ])

In [ ]:
# MULTILABEL CLASSIFICATION


from sklearn.neighbors import KNeighborsClassifier
y_train_large = (y_train >= 7)
y_train_odd = (y_train % 2 == 1)
y_multilabel = np.c_[y_train_large, y_train_odd]
knn_clf = KNeighborsClassifier()
knn_clf.fit(X_train, y_multilabel)
# This code creates a y_multilabel array containing two target labels for each digit image: the first
# indicates whether or not the digit is large (7, 8, or 9) and the second indicates whether or not it is odd.
# The next lines create a KNeighborsClassifier instance (which supports multilabel classification, but
# not all classifiers do) and we train it using the multiple targets array. Now you can make a prediction, and
# notice that it outputs two labels:
knn_clf.predict([some_digit])


# here are many ways to evaluate a multilabel classifier, and selecting the right metric really depends on
# your project. For example, one approach is to measure the F 1 score for each individual label (or any other
# binary classifier metric discussed earlier), then simply compute the average score. This code computes
# the average F 1 score across all labels:
y_train_knn_pred = cross_val_predict(knn_clf, X_train, y_train, cv=3)
f1_score(y_train, y_train_knn_pred, average="macro")

# This assumes that all labels are equally important, which may not be the case. In particular, if you have
# many more pictures of Alice than of Bob or Charlie, you may want to give more weight to the classifier’s
# score on pictures of Alice. One simple option is to give each label a weight equal to its support (i.e., the
# number of instances with that target label). To do this, simply set average="weighted" in the preceding
# code. 

In [ ]:
# CLEARING NOISE FROM IMAGES


# Let’s start by creating the training and test sets by taking the MNIST images and adding noise to their pixel
# intensities using NumPy’s randint() function. The target images will be the original images:
noise = np.random.randint(0, 100, (len(X_train), 784))
X_train_mod = X_train + noise
noise = np.random.randint(0, 100, (len(X_test), 784))
X_test_mod = X_test + noise
y_train_mod = X_train
y_test_mod = X_test

# let’s train the classifierand make it clean this image:
knn_clf.fit(X_train_mod, y_train_mod)
clean_digit = knn_clf.predict([X_test_mod[some_index]])
plot_digit(clean_digit)